In [1]:
%store -r affy_platform_to_geo

In [2]:
import sys
sys.path.append('/home/ubuntu/lib/')

In [ ]:
! ls /home/ubuntu/lib/

In [3]:
from metadb.api import ElasticApi

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es = Elasticsearch()
api = ElasticApi(es, 
                 samples_index='samples_dev',
                 series_index='series_dev'
                )

In [ ]:
?api.update_by_query

In [ ]:
len(affy_platform_to_geo)

In [6]:
from metadb.api import utils
from elasticsearch.helpers import bulk

In [19]:
def update_platforms():
    body = {
        "query": {
            "filtered": {
                "filter": {
                    "term": {
                        "data_source": 'array-express'
#                         "platforms": 'A-AFFY-44'
                    }
                }
            }
        },
        "_source": ['platforms']
    }
    
    series = [(s['_id'], s['_source']) for s in es.search(index='series_dev', body=body, size=10000)['hits']['hits']]
    for _id, s in series:
        s['platforms'] = [
            p if p not in affy_platform_to_geo else affy_platform_to_geo[p]
            for p in s['platforms']
        ]
        
    print(len(series))
#     print([s['platforms'] for _id, s in series][:10])
    
    for bucket in utils.iter_bucket(series, 1000):
        actions = [
                {
                    '_op_type': 'update',
                    '_index': 'series_dev',
                    '_type': 'series_dev',
                    '_id': _id,
                    'doc': {
                        'platforms': x['platforms']
                    }
                }
                for (_id, x) in bucket
            ]
#         print(actions)
#         break
        res = bulk(es, actions)
        print(res)


In [20]:
update_platforms()

9038
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(1001, [])
(29, [])


In [ ]:
for ae_platform, geo_platform in affy_platform_to_geo.items():
    print(ae_platform, geo_platform)
    update_platform(ae_platform, geo_platform)
    

In [ ]:
[s['accession'] for s in api.search_samples(fields='accession', platform='A-AFFY-44')][:10]

In [ ]:
affy_platform_to_geo['A-AFFY-44']

In [ ]:
update_platform('A-AFFY-44', 'GPL570')